In [3]:
# ensure that lists are read as list and not as strings
import pandas as pd
from tqdm import tqdm # progress apply :)
tqdm.pandas()
from datetime import datetime
today=f"{datetime.now():%Y-%m-%d}"

today

'2025-06-02'

In [177]:
def import_survey(excel_path, sheet_name):
    """
    Load and preprocess the survey draft Excel file.
    Forward fills Sections and cleans up columns.
    Returns a DataFrame ready for parsing.
    """
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    df['Sections'] = df['Sections'].ffill()
    # Optional: strip whitespace from string columns
    for col in ['Sections', 'Label A', 'Label B', 'Question Type', 'SoM', 'Survey Question', 'Choices']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    df.dropna(inplace=True)

    return df


import_survey("2025-05-28 SurveyDraft.xlsx",sheet_name="Survey Questions")

,Sections,Label A,Label B,Question Type,SoM,Survey Question,Choices
0,1. Personal impression of the crisis,Location & Setting,nan,TE:Essay,OPEN,Can you name one specific example where the en...,nan
1,1. Personal impression of the crisis,Personal affect,nan,MC:SingleAnswer,LIKERT,"On a scale of 1 to 5, to what extent did the e...",Not at all; To a small extent; To a moderate e...
2,1. Personal impression of the crisis,nan,nan,Matrix,nan,To what extent do you agree or disagree with t...,Strongly disagree; Disagree; Neither agree nor...
3,1. Personal impression of the crisis,Conflict Focus,Financial housholds,Matrix:Answer,LIKERT,The energy crisis has significantly increased ...,nan
4,1. Personal impression of the crisis,Conflict Focus,Vulnerable people,Matrix:Answer,LIKERT,The energy crisis has impacted low-income hous...,nan
...,...,...,...,...,...,...,...
94,Textboxes,Introduction to the Survey,nan,nan,nan,"Dear participant, Thank you for your willingne...",nan
95,Textboxes,Motivation causes (open):,nan,nan,nan,"""Understanding who or what shaped the causes o...",nan
96,Textboxes,Motivation responses (open):,nan,nan,nan,"""Now let’s focus on how the energy crisis was ...",nan
97,Textboxes,Motivation moral (open):,nan,nan,nan,"""Every crisis teaches a lesson. The energy cri...",nan


In [179]:
def write_qualtrics_file(df, output_path):
    """
    Write the entire Qualtrics TXT file.
    - Groups questions by 'Sections' (blocks).
    - Calls `write_block` for each block.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        # Write the advanced format header
        f.write('[[AdvancedFormat]]\n\n')

        # Group questions by block and write each block
        for block, block_df in df.groupby('Sections'):
            write_block(f, block, block_df)

    print(f"Qualtrics import file written to: {output_path}")


In [181]:

def write_block(f, block_name, block_df):
    """
    Write a block header and all questions in the block.
    - Calls `write_question` for each question in the block.
    """
    # Write the block header
    f.write(f'[[Block:{block_name}]]\n')

    # Write each question in the block
    for _, row in block_df.iterrows():
        write_question(f, row, block_df)
        f.write('\n')  # Blank line between questions

In [231]:
def write_question(f, row,block_df=None):
    """
    Write a single question based on its type.
    - Handles different question types (e.g., TE, MC, Matrix).
    """

    qtype = row.get('Question Type', '')
    question_text = row.get('Survey Question', '').strip()
    choices_raw = row.get('Choices', '').strip()
    label_a = row.get('Label A', '').strip()
    label_b = row.get('Label B', '').strip()

    # Skip rows without a valid question type or question text
    if not qtype or not question_text:
        return  # Skip this row
        
    # Generate QID based on Label A and Label B
    qid = generate_qid(label_a, label_b)

    # Handle matrix questions separately
    if qtype == 'Matrix':
        # Extract matrix statements (rows with 'Matrix:Answer')
        matrix_statements = block_df[block_df['Question Type'] == 'Matrix:Answer']['Survey Question'].tolist()
        
        
        # Extract Likert scale options from the 'Choices' column of the matrix question row
        scale_points = [choice.strip() for choice in choices_raw.split(';') if choice.strip()]

        # Pass the matrix question text, statements, and scale points to the matrix writer
        write_matrix_question(f, question_text, matrix_statements, scale_points)
        return

    # Map question type and subtype
    qualtrics_qtype, qualtrics_subtype = map_question_type(qtype)
    
    # Write the question header
    if qualtrics_subtype in ['MC', 'CS']:
        f.write(f'[[Question:{qualtrics_qtype}:{qualtrics_subtype}]]\n')
        #f.write(f'[[ID:{qid}]]\n')
        f.write(f"{question_text}\n")
        write_choices(f, choices_raw)
    elif qualtrics_qtype in ['MC', 'CS']:
        f.write(f'[[Question:{qualtrics_qtype}]]\n')
       # f.write(f'[[ID:{qid}]]\n')
        f.write(f"{question_text}\n")
        write_choices(f, choices_raw)
    elif qualtrics_qtype in ['TE']:
        print(qualtrics_subtype)
        f.write(f'[[Question:{qualtrics_qtype}:{qualtrics_subtype}]]\n')
       # f.write(f'[[ID:{qid}]]\n')
        f.write(f"{question_text}\n")
    elif qualtrics_qtype in ['DB']:
        f.write(f'[[DB]]\n')
        f.write(f"{question_text}\n")
        
    



def generate_qid(label_a, label_b):
    """
    Generate a unique QID based on Label A and Label B.
    - Combines Label A and Label B with an underscore.
    - If Label B is missing, only Label A is used.

    Parameters:
        label_a (str): The main label for the question.
        label_b (str): The sublabel for the question.

    Returns:
        str: The generated QID.
    """
    if label_a and label_b:
        return f"{label_a}_{label_b}".replace(" ", "").replace(",", "").replace(".", "")
    elif label_a:
        return label_a.replace(" ", "").replace(",", "").replace(".", "")
    else:
        return "UnnamedQuestion"




def map_question_type(qtype):
    """
    Map the survey Question Type to Qualtrics question type and subtype.
    """
    qtype = qtype.upper()

    if 'TE' in qtype:
        if 'ESSAY' in qtype:
            return 'TE', 'Essay'
        else:
            return 'TE', 'SingleLine'
    elif 'MC' in qtype:
        if "SingleAnswer" in qtype:
            return 'MC', 'SingleAnswer'
        if "MultipleAnswer" in qtype:
            return 'MC', 'MultipleAnswer'
        else:
            return 'MC', 'SingleAnswer'
    elif 'MATRIX' in qtype:
        return 'Matrix', None
    elif 'CONSTANTSUM' in qtype or 'CS' in qtype:
        return 'CS', None
    elif "DB" in qtype:
        return "DB",None
    else:
        # Default fallback
        return None, None

def write_choices(f, choices_raw):
    """
    Write choices for MC or CS questions.
    - Choices are semicolon-separated in the Excel file.
    """
    if not choices_raw:
        return
    choices = [c.strip() for c in choices_raw.split('; ') if c.strip()]
    
    if choices:
        f.write('[[Choices]]\n')
        for choice in choices:
            f.write(f"{choice}\n")
        f.write('\n')


def write_matrix_question(f, question_text, statements, scale_points):
    """
    Write a matrix question to the Qualtrics TXT file.
    - The main matrix question text is passed as `question_text`.
    - Statements are passed as a list of strings.
    - Likert scale options are passed as a list of strings.

    Parameters:
        f (file object): The open file to write to.
        question_text (str): The main matrix question text.
        statements (list): A list of matrix statements.
        scale_points (list): A list of Likert scale options.
    """
    # Write the matrix question header
    f.write(f"[[Question:Matrix]]\n")
    f.write(f"{question_text}\n")

    # Write the matrix statements under [[Choices]]
    f.write("[[Choices]]\n")
    for statement in statements:
        f.write(f"{statement.strip()}\n")
    f.write("\n")

    # Write the Likert scale options under [[Answers]]
    f.write("[[Answers]]\n")
    for scale in scale_points:
        f.write(f"{scale}\n")
    f.write("\n")



# Import the survey draft
excel_path = "2025-05-28 SurveyDraft.xlsx"
sheet_name = "Survey Questions"
output_path = "qualtrics_import.txt"


# Step 1: Import the survey
df = import_survey(excel_path, sheet_name)

# Step 2: Write the Qualtrics TXT file

write_qualtrics_file(df, output_path)

Essay
Essay
Essay
Essay
Essay
Essay
Essay
Qualtrics import file written to: qualtrics_import.txt


Essay
Essay
Essay
Essay
Essay
Essay
Essay
Qualtrics import file written to: qualtrics_import.txt
